In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import seaborn as sns

from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

#from workalendar.europe import Russia
from collections import Counter
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import RFE
from xgboost import XGBRegressor
from scipy.stats import zscore


In [ ]:
##############################################
df = pd.read_excel("data.xlsx", parse_dates=['Date'])
df['Day_of_week'] = df['Date'].dt.dayofweek # 0 - пнд ... 6 - вс

# маски
full_zero_mask = (df['Income'] == 0) & (df['Outcome'] == 0) & (df['Balance'] == 0)
weekend_mask = (df['Day_of_week'] == 5) | (df['Day_of_week'] == 6)

full_zero_indices = df[full_zero_mask].index.tolist() # три нуля
weekend_zero_indices = df[full_zero_mask & weekend_mask].index.tolist() # выходные нули
weekday_zero_indices = df[full_zero_mask & ~weekend_mask].index.tolist() # будние нули

min_year = df['Date'].dt.year.min()
max_year = df['Date'].dt.year.max()
cal = Russia()
all_holidays = []

for year in range(min_year, max_year + 1):
    all_holidays.extend(cal.holidays(year))

# df с праздниками
holidays_df = pd.DataFrame(all_holidays, columns=['date', 'holiday_name'])
holidays_df['date'] = pd.to_datetime(holidays_df['date'])


weekday_zero_dates = df.loc[weekday_zero_indices, 'Date']

# добавим признак праздника в основном DataFrame
df['is_holiday'] = df['Date'].isin(holidays_df['date']).astype(int)

# проверяем пересечения с нулевыми буднями
holiday_zeros = weekday_zero_dates[weekday_zero_dates.isin(holidays_df['date'])]

holiday_mask = df['Date'].isin(holidays_df['date'])

anomalous_zeros_mask = full_zero_mask & ~weekend_mask & ~holiday_mask
anomalous_indices = df[anomalous_zeros_mask].index
df.loc[anomalous_indices, 'is_holiday'] = 1


In [ ]:
##############################################
# USD_RUB
df_usd = pd.read_excel("usd.xlsx",  parse_dates=['data'])

df = df.merge(
    df_usd[['data', 'curs']],
    left_on='Date',
    right_on='data',
    how='left'
)

df = df.drop(columns=['data'])
df = df.rename(columns={'curs': 'USD_RUB'})
df['USD_RUB'] = df['USD_RUB'].bfill()  # первым известным значением

In [ ]:
###########################################
inflation_df = pd.read_excel("inflation.xlsx", parse_dates=['Дата'])
inflation_df['Дата'] = '01.' + inflation_df['Дата']  # Добавляем первое число
inflation_df['Дата'] = pd.to_datetime(inflation_df['Дата'], format='%d.%m.%Y')  # Преобразуем в дату

df = df.merge(
    inflation_df[['Дата', 'Ключевая ставка, % годовых', 'Инфляция, % г/г']],
    left_on='Date',
    right_on='Дата',
    how='left'
)

df = df.drop(columns=['Дата'])
df = df.rename(columns={'Ключевая ставка, % годовых': 'Key_rate', 'Инфляция, % г/г': 'Inflation'})
df['Key_rate'] = df['Key_rate'].bfill().ffill()
df['Inflation'] = df['Inflation'].bfill().ffill()

In [ ]:
##############################################
# Обработка выбросов
z_scores = zscore(df['Balance'])
df = df[(np.abs(z_scores) < 3)]  # Удаляем выбросы за пределами 3σ

In [ ]:
##############################################

test_size = int(len(df) * 0.2)
df_train, df_test = df.iloc[:-test_size], df.iloc[-test_size:]
# добавляем фичи
def add_features(df, target='Balance'):
    """Создание временных признаков"""
    df = df.copy()
    for lag in [1, 2, 3, 7, 14, 30, 60]:
        df[f'{target}_lag_{lag}'] = df[target].shift(lag)

    for window in [3, 7, 14, 30]:
        df[f'{target}_rolling_mean_{window}'] = df[target].rolling(window).mean().shift(1)
        df[f'{target}_rolling_std_{window}'] = df[target].rolling(window).std().shift(1)

    df['day_of_week'] = df['Date'].dt.dayofweek
    df['month'] = df['Date'].dt.month
    df['quarter'] = df['Date'].dt.quarter
    df['year'] = df['Date'].dt.year
    df['day_of_year'] = df['Date'].dt.dayofyear
    return df.fillna(0)

df_train = add_features(df_train)
df_test = add_features(df_test)

X_train, X_test = df_train.drop(columns=['Balance', 'Date']), df_test.drop(columns=['Balance', 'Date'])
y_train, y_test = df_train['Balance'], df_test['Balance']

X_train
def feature_selection(X_train, y_train, n_features_to_select= 10, importance_threshold= 0.1):

# 1 --- фильтрационный метод (отбор по корреляции)
    corr_matrix = X_train.corr()
    high_corr_features = set()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > 0.8:  # Порог корреляции
                colname = corr_matrix.columns[i]
                high_corr_features.add(colname)

    X_train = X_train.drop(columns=high_corr_features)
    print(high_corr_features)

# 2 --- встроенный метод, Лассо
    lasso = LassoCV(cv=5, random_state=42)
    lasso.fit(X_train, y_train)

    selected_features_lasso = X_train.columns[lasso.coef_ != 0]

# 3 --- оберточные

# 3.1 --- RFE

    # Инициализация модели и селектора
    model = XGBRegressor(random_state=42)
    selector = RFE(estimator=model,
                   n_features_to_select=n_features_to_select,
                   step=1)

    # Обучение селектора
    selector.fit(X_train, y_train)

    # Получение отобранных признаков
    selected_features_rfe = X_train.columns[selector.support_]


# 3.2 ---   Анализ важности признаков с помощью XGBoost
    print("\n" + "="*50)
    print("Анализ важности признаков")
    print("="*50)

    # Обучение модели для анализа важности
    xgb = XGBRegressor(random_state=42)
    xgb.fit(X_train, y_train)

    # Получение важности признаков
    importance = xgb.feature_importances_
    feature_importance = pd.Series(importance, index=X_train.columns)


    # Отбор признаков по порогу важности
    selected_features_xgb = feature_importance[feature_importance > importance_threshold].index

# 4 --- итог
    # Собираем результаты всех методов
    all_selections = {
        'RFE': set(selected_features_rfe),
        'Lasso': set(selected_features_lasso),
        'XGBoost': set(selected_features_xgb)
    }

    # Анализ пересечений
    feature_votes = Counter()
    for method, features in all_selections.items():
        feature_votes.update(features)

    # Выбираем признаки, отобранные хотя бы двумя методами
    final_features = [feature for feature, count in feature_votes.items() if count >= 2]
    print("\nФинальный отбор признаков:", final_features)

    # Визуализация согласованности методов
    pd.DataFrame({
        'Признак': list(feature_votes.keys()),
        'Количество методов': list(feature_votes.values())
    }).sort_values('Количество методов', ascending=False).plot.barh(x='Признак', y='Количество методов')
    plt.xticks(range(0, 5))
    plt.title('Согласованность методов отбора')
    plt.show()

    return final_features
    ff = feature_selection(X_train, y_train)

In [ ]:
from sklearn.feature_selection import mutual_info_regression

# Отбор по взаимной информации
def select_features_mi(X, y, threshold=0.05):
    mi = mutual_info_regression(X, y)
    selected = X.columns[mi > threshold]
    return selected


ff = select_features_mi(X_train, y_train)

# Загрузка и подготовка данных
X_train= X_train[ff]  # Используем предварительно отобранные фичи
X_test= X_test[ff]

In [ ]:
# 2. Настройка кросс-валидации (адаптированный подход из первой реализации)
tscv = TimeSeriesSplit(n_splits=5)

# 3. Оптимизированный пайплайн (лучшее из обеих реализаций)
pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', StandardScaler()),
    ('model', XGBRegressor(
        objective='reg:squarederror',
        random_state=52,
        n_jobs=-1,
    ))
])

# 4. Умеренный GridSearch
param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [3, 4],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__subsample': [0.8, 0.9],
    'model__reg_alpha': [0.1, 1],
    'model__reg_lambda': [0.1, 1]
}

# 5. Безопасный GridSearch только на тренировочных данных
grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=tscv,
    scoring='r2',
    verbose=1,
    n_jobs=-1
)

# Обучение с валидацией
grid.fit(X_train, y_train)

# Лучшая модель
best_model = grid.best_estimator_

# Прогноз и оценка
y_pred = best_model.predict(X_test)

# Визуализация (улучшенный вариант)
plt.figure(figsize=(14, 6))
plt.plot(y_test.values, label='Фактические значения', color='#1f77b4', linewidth=2)
plt.plot(y_pred, label='Прогноз', color='#ff7f0e', linestyle='--', linewidth=2)
plt.title('Прогноз vs Факт: временной ряд', fontsize=14, pad=15)
plt.xlabel('Временной индекс', fontsize=12)
plt.ylabel('Balance', fontsize=12)
plt.legend(fontsize=12, framealpha=1, shadow=True)
plt.grid(True, linestyle=':', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Метрики
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print('=== Результаты ===')
print(f'Лучшие параметры: {grid.best_params_}')
print(f'MAE:  {mae:.3f}')
print(f'RMSE: {rmse:.3f}')
print(f'R²:   {r2:.3f}')
print(f'Использовано признаков: {len(ff)}')